### load the data

In [34]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import cufflinks as cf

In [35]:
cf.go_offline()

In [36]:
train = pd.read_csv('covid.csv', low_memory=False)

In [37]:
train.head()

,Id,County,Province_State,Country_Region,Population,Weight,Date,Target,TargetValue
0,1,NaN,NaN,Afghanistan,27657145,0.058359,2020-01-23,ConfirmedCases,0
1,2,NaN,NaN,Afghanistan,27657145,0.583587,2020-01-23,Fatalities,0
2,3,NaN,NaN,Afghanistan,27657145,0.058359,2020-01-24,ConfirmedCases,0
3,4,NaN,NaN,Afghanistan,27657145,0.583587,2020-01-24,Fatalities,0
4,5,NaN,NaN,Afghanistan,27657145,0.058359,2020-01-25,ConfirmedCases,0


### zoom into different countries:USA

In [38]:
usa = train[train['Country_Region']=='US']

In [39]:
usa.isnull().sum()

Id                    0
County            15400
Province_State      280
Country_Region        0
Population            0
Weight                0
Date                  0
Target                0
TargetValue           0
dtype: int64

In [51]:
usa[usa['Province_State'].isnull()]['Population'].unique() # that means the data for one state is missing

array([324141489])

In [70]:
usa[usa['Population']==324141489]['Province_State'].unique()

array([nan], dtype=object)

In [54]:
usa_df = usa[['Date', 'Province_State', 'Target', 'TargetValue']]

In [62]:
usa_df1 = usa_df.groupby(['Province_State','Target']).sum().unstack('Target').reset_index()

In [72]:
usa_df1.columns=['states', 'confirmed', 'fatalities']

In [136]:
state_abbr = pd.read_csv('states.csv')

In [138]:
state_abbr.columns=['states', 'abbr']

In [139]:
usa_df2=state_abbr.merge(usa_df1, on='states')

In [146]:
fig = px.choropleth(usa_df2, locations='abbr', locationmode="USA-states",scope='usa',
                    color='confirmed', hover_data=['states', 'fatalities'])
fig.show()

### zoom into different countries:USA

In [147]:
Australia = train[train['Country_Region']=='Australia']

In [149]:
Australia.isnull().sum()

Id                   0
County            2520
Province_State     280
Country_Region       0
Population           0
Weight               0
Date                 0
Target               0
TargetValue          0
dtype: int64

In [150]:
Aus_df = Australia[['Date', 'Province_State', 'Target', 'TargetValue']]

In [209]:
Aus_df1 = Aus_df.groupby(['Date','Province_State','Target']).sum().unstack('Target').reset_index()

In [211]:
Aus_df1.columns=['date','states', 'confirmed', 'fatalities']

In [217]:
total_confirmed_Aus = Aus_df1.groupby(['date', 'states']).sum()['confirmed'].unstack('states').reset_index().set_index('date').cumsum().reset_index()

In [224]:
total_confirmed_Aus.head()

,date,states,value
0,2020-01-23,Australian Capital Territory,0
1,2020-01-24,Australian Capital Territory,0
2,2020-01-25,Australian Capital Territory,0
3,2020-01-26,Australian Capital Territory,0
4,2020-01-27,Australian Capital Territory,0


In [220]:
total_confirmed_Aus = pd.melt(total_confirmed_Aus, id_vars=['date'], value_vars=total_confirmed_Aus.columns)

In [221]:
total_confirmed_vAus

,date,states,value
0,2020-01-23,Australian Capital Territory,0
1,2020-01-24,Australian Capital Territory,0
2,2020-01-25,Australian Capital Territory,0
3,2020-01-26,Australian Capital Territory,0
4,2020-01-27,Australian Capital Territory,0
...,...,...,...
1115,2020-06-06,Western Australia,599
1116,2020-06-07,Western Australia,599
1117,2020-06-08,Western Australia,599
1118,2020-06-09,Western Australia,599


In [225]:
fig = px.line(total_confirmed_Aus, x='date', y='value', color='states', title='Covid cofirmed cases till 2020-06-10 in Australia')
fig.show()